In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# REF: https://www.kaggle.com/bustam/cnn-in-keras-for-kannada-digits

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
IW = 28
IH = 28
TP = 10
seed = 0
np.random.seed(seed)

class MNISTLoader():
    def __init__(self):
        
        root = '/kaggle/input/Kannada-MNIST/'
        sample_file = os.path.join(root, 'sample_submission.csv')
        dev_hard_file = os.path.join(root, 'Dig-MNIST.csv')
        train_file = os.path.join(root, 'train.csv')
        test_file = os.path.join(root, 'test.csv')

        for file in [dev_hard_file, sample_file, train_file, test_file]:
            assert os.path.exists(file), 'Please download dataset and save to "data/Kannada-MNIST/" before boot'

        X_train_all, Y_train_all = self.read_csv(train_file, type='train')
        train_pers, dev_pers = self.random_divide(len(X_train_all), props=[0.95, 0.05])
        self.X_train, self.Y_train = X_train_all[train_pers], Y_train_all[train_pers]
        self.X_dev, self.Y_dev = X_train_all[dev_pers], Y_train_all[dev_pers]
        self.X_dev_hard, self.Y_dev_hard = self.read_csv(dev_hard_file, type='dev_hard')
        self.X_test, _ = self.read_csv(test_file, type='test')
        self.num_train_data, self.num_dev_data, self.num_test_data = self.X_train.shape[0], self.X_dev.shape[0], self.X_test.shape[0]

    @staticmethod
    def read_csv(file_path, type='train'):
        """ 读取 csv 数据 """
        
        data = pd.read_csv(file_path)
        X = data.iloc[: ,1: ].values
        Y = data.iloc[: ,0].values 
        
        X = X.reshape((X.shape[0], IH, IW, 1))
        Y = np.eye(TP)[Y] if type != 'test' else None
        return X, Y
    
    @staticmethod
    def random_divide(data_size, props):
        """ 返回将数据按 props 切分后的索引 """
        
        assert sum(props) == 1
        permutation = list(np.random.permutation(data_size))
        return [permutation[int(data_size * sum(props[: i])): int(data_size * sum(props[: i+1]))] for i in range(len(props))]

    def random_mini_batches(self, batch_size = 64):
        """ 切分训练集为 mini_batch """
        
        data_size = len(self.X_train)
        permutation = list(np.random.permutation(data_size))
        batch_permutation_indices = [permutation[i: i + batch_size] for i in range(0, data_size, batch_size)]
        for batch_permutation in batch_permutation_indices:
            yield self.X_train[batch_permutation], self.Y_train[batch_permutation]
            
data_loader = MNISTLoader()

In [ ]:
class CNN(tf.keras.Model):
    """ CNN 模型 """
    def __init__(self, dropout_rate=0.1):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[3, 3],
            strides=1,
            padding='same',
            activation=tf.nn.relu
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[3, 3],
            strides=1,
            padding='same',
            activation=tf.nn.relu
        )
        #         self.bn2 = tf.keras.layers.BatchNormalization()
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dropout1 = tf.keras.layers.Dropout(rate=dropout_rate)
        self.dense2 = tf.keras.layers.Dense(units=128, activation=tf.nn.relu)
        self.dropout2 = tf.keras.layers.Dropout(rate=dropout_rate)
        self.dense3 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        #         x = self.bn2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.dropout2(x)
        x = self.dense3(x)
        output = tf.nn.softmax(x)
        return output

In [ ]:
class Xception(tf.keras.Model):
    """ Xception 模型 """
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=16,
            kernel_size=[3, 3],
            padding='same',
            activation=tf.nn.relu
        )
        self.separable_conv2 = tf.keras.layers.SeparableConv2D(
            filters=32,
            kernel_size=[3, 3],
            padding='same'
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2)
        self.separable_conv3 = tf.keras.layers.SeparableConv2D(
            filters=64,
            kernel_size=[3, 3],
            padding='same'
        )
        self.pool3 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2)
        self.separable_conv4 = tf.keras.layers.SeparableConv2D(
            filters=64,
            kernel_size=[3, 3],
            padding='same'
        )
        self.separable_conv5 = tf.keras.layers.SeparableConv2D(
            filters=64,
            kernel_size=[3, 3],
            padding='same'
        )
        self.separable_conv6 = tf.keras.layers.SeparableConv2D(
            filters=64,
            kernel_size=[3, 3],
            padding='same'
        )
        self.pool6 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2)
        self.separable_conv7 = tf.keras.layers.SeparableConv2D(
            filters=64,
            kernel_size=[3, 3],
            padding='same'
        )
        self.separable_conv8 = tf.keras.layers.SeparableConv2D(
            filters=64,
            kernel_size=[3, 3],
            padding='same'
        )
        self.separable_conv9 = tf.keras.layers.SeparableConv2D(
            filters=64,
            kernel_size=[3, 3],
            padding='same'
        )
        self.pool9 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2)
        self.separable_conv10 = tf.keras.layers.SeparableConv2D(
            filters=10,
            kernel_size=[3, 3],
            padding='same'
        )
        
        #         self.dw_conv = tf.keras.layers.DepthwiseConv2D(
        #             kernel_size=(7, 7),
        #             strides=(1, 1),
        #             padding='valid',
        #         )
        self.global_average_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.flatten = tf.keras.layers.Flatten()
        #         self.dense = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.separable_conv2(x)
        x = self.pool2(x)
        x = self.separable_conv3(x)
        x = self.pool3(x)
        res = x
        x = self.separable_conv4(x)
        x = self.separable_conv5(x)
        x = self.separable_conv6(x)
        x += res
        res = x
        x = self.separable_conv7(x)
        x = self.separable_conv8(x)
        x = self.separable_conv9(x)
        x += res
        x = self.pool9(x)
        x = self.separable_conv10(x)
        #         x = self.dw_conv(x)
        x = self.global_average_pool(x)
        x = self.flatten(x)
        #         x = self.dense(x)
        output = tf.nn.softmax(x)
        return output

In [ ]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        assert units % 32 == 0
        self.k = units // 32
        self.activation = activation

    def build(self, input_shape):
        
        k = self.k
        self.conv_1_1 = tf.keras.layers.Conv2D(
            filters=8*k,
            kernel_size=[1, 1],
            strides=1,
            padding='same'
        )
        self.conv_1_1_t3 = tf.keras.layers.Conv2D(
            filters=12*k,
            kernel_size=[1, 1],
            strides=1,
            padding='same'
        )
        self.conv_3_3 = tf.keras.layers.Conv2D(
            filters=16*k,
            kernel_size=[3, 3],
            strides=1,
            padding='same'
        )
        self.conv_1_1_t5 = tf.keras.layers.Conv2D(
            filters=k,
            kernel_size=[1, 1],
            strides=1,
            padding='same'
        )
        self.conv_3_3_t5 = tf.keras.layers.Conv2D(
            filters=2*k,
            kernel_size=[3, 3],
            strides=1,
            padding='same'
        )
        self.conv_5_5 = tf.keras.layers.Conv2D(
            filters=4*k,
            kernel_size=[3, 3],
            strides=1,
            padding='same'
        )
        self.pool_t = tf.keras.layers.MaxPool2D(
            pool_size=[3, 3],
            strides=1,
            padding='same'
        )
        self.pool = tf.keras.layers.Conv2D(
            filters=4*k,
            kernel_size=[1, 1],
            strides=1,
            padding='same'
        )
        self.bn = tf.keras.layers.BatchNormalization()

    def call(self, inputs):
        x_1_1 = self.conv_1_1(inputs)
        x_1_1_t3 = self.conv_1_1_t3(inputs)
        x_3_3 = self.conv_3_3(x_1_1_t3)
        x_1_1_t5 = self.conv_1_1_t5(inputs)
        x_3_3_t5 = self.conv_3_3_t5(x_1_1_t5)
        x_5_5 = self.conv_5_5(x_3_3_t5)
        x_pool_t = self.pool_t(inputs)
        x_pool = self.pool(x_pool_t)
        x = tf.concat([x_1_1, x_3_3, x_5_5, x_pool], axis=-1)
        x = self.bn(x)
        if self.activation is not None:
            x = self.activation(x)
        return x


class Inception(tf.keras.Model):
    """ Inception 模型 """
    def __init__(self, dropout_rate=0.1):
        super().__init__()
        self.layer1_1 = InceptionModule(units=32)
        self.layer1_2 = InceptionModule(units=32)
        self.layer1_3 = InceptionModule(units=64, activation=tf.nn.relu)
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2)
        self.layer2_1 = InceptionModule(units=64)
        self.layer2_2 = InceptionModule(units=64)
        self.layer2_3 = InceptionModule(units=64, activation=tf.nn.relu)
        self.layer3_1 = InceptionModule(units=64)
        self.layer3_2 = InceptionModule(units=64)
        self.layer3_3 = InceptionModule(units=64, activation=tf.nn.relu)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2)
        self.layer4_1 = InceptionModule(units=128)
        self.layer4_2 = InceptionModule(units=128)
        self.layer4_3 = InceptionModule(units=64, activation=tf.nn.relu)
        self.pool4 = tf.keras.layers.MaxPool2D(pool_size=[3, 3], strides=2)
        self.conv5 = tf.keras.layers.Conv2D(
            filters=10,
            kernel_size=[1, 1],
            padding='same'
        )
        self.global_average_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.flatten = tf.keras.layers.Flatten()

    def call(self, inputs):
        x = inputs
        x = self.layer1_1(x)
        x = self.layer1_2(x)
        x = self.layer1_3(x)
        x = self.pool1(x)
        res = x
        x = self.layer2_1(x)
        x = self.layer2_2(x)
        x = self.layer2_3(x)
        x += res
        res = x
        x = self.layer3_1(x)
        x = self.layer3_2(x)
        x = self.layer3_3(x)
        x += res
        x = self.pool2(x)
        res = x
        x = self.layer3_1(x)
        x = self.layer3_2(x)
        x = self.layer3_3(x)
        x += res
        res = x
        x = self.layer4_1(x)
        x = self.layer4_2(x)
        x = self.layer4_3(x)
        x += res
        x = self.pool4(x)
        x = self.conv5(x)
        x = self.global_average_pool(x)
        x = self.flatten(x)
        output = tf.nn.softmax(x)
        return output

In [ ]:
num_epochs = 30
batch_size = 50
print_step = 1000
dev_step = 100000
learning_rate = 2e-3

In [ ]:
# model = CNN(dropout_rate=0.5)
# model = Xception()
model = Inception()
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

In [ ]:
def one_hots_to_labels(one_hots):
    return np.array([[np.argmax(one_hot)] for one_hot in one_hots])

def predict(model, data_loader, batch_size=10000, type='train'):
    wrongs = 0
    data_length = data_loader.num_train_data if type == 'train' else data_loader.num_dev_data
    X_map = {"train": data_loader.X_train, "dev": data_loader.X_dev, "dev_hard": data_loader.X_dev_hard}
    Y_map = {"train": data_loader.Y_train, "dev": data_loader.Y_dev, "dev_hard": data_loader.Y_dev_hard}
    X = X_map[type]
    Y = Y_map[type]
    for i in range(0, data_length, batch_size):
        print(f'predict {i}', end='\r')
        X_batch = X[i: i + batch_size] / 255.
        Y_batch = Y[i: i + batch_size]
        Y_batch_= model.predict(X_batch)
        Y_batch, Y_batch_ = one_hots_to_labels(Y_batch), one_hots_to_labels(Y_batch_)
        mask = Y_batch.reshape((Y_batch.shape[0], )) - Y_batch_.reshape(Y_batch_.shape[0], )
        wrongs += len(np.flatnonzero(mask))
    return 1- wrongs / data_length

print(predict(model, data_loader, type='dev'))

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 10,
                                   width_shift_range = 0.25,
                                   height_shift_range = 0.25,
                                   shear_range = 0.1,
                                   zoom_range = 0.25,
                                   horizontal_flip = False)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
@tf.function
def train_one_step(X, Y):
    with tf.GradientTape() as tape:
        Y_ = model(X)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
            y_true=Y,
            y_pred=Y_
        ))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    return loss


for i in range(num_epochs):
# for i in range(2):
    num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
    for j, (X, Y) in enumerate(train_datagen.flow(data_loader.X_train, data_loader.Y_train, batch_size=batch_size)):
        loss = train_one_step(X, Y)
        if (i * data_loader.num_train_data + j * batch_size) % print_step == 0:
            print(f"{i} - {j * batch_size: 6}: loss {loss.numpy()}")
        if (i * data_loader.num_train_data + j * batch_size) % dev_step == 0:
            train_accuracy = predict(model, data_loader, type='train')
            dev_accuracy = predict(model, data_loader, type='dev')
            dev_hard_accuracy = predict(model, data_loader, type='dev_hard')
            print(f'train accuracy: {train_accuracy: .2%} dev accuracy: {dev_accuracy: .2%} dev hard accuracy: {dev_hard_accuracy: .2%}')

In [ ]:
print(predict(model, data_loader, type='train'))
print(predict(model, data_loader, type='dev'))
print(predict(model, data_loader, type='dev_hard'))

In [ ]:
# def write_csv(file_path, labels):
#     csv_list = [['id', 'label']]
#     for i, label in enumerate(labels):
#         csv_list.append([str(i), str(label)])
#     csv_str = ''
#     for line_list in csv_list:
#         csv_str += ','.join(line_list) + '\n'
#     with open(file_path, 'w') as f:
#         f.write(csv_str)

In [ ]:
Y_test_ = one_hots_to_labels(model.predict(data_loader.X_test / 255.))
labels = Y_test_.reshape((Y_test_.shape[0], ))

In [ ]:
# write_csv('/kaggle/working/submission.csv', labels)

In [ ]:
submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission['label'] = labels
submission.head()
submission.to_csv("submission.csv",index=False)